In [ ]:
##############################################################
# Script     : not_core.ipynb
# Author     : Luis Raúl Figueroa Martínez
# Date       : 11/06/2025
# Description:
#     Este script analiza pangenomas de las bacterias de CAMDA 2024 
#     y CAMDA20 25 juntas para identificar
#     familias de genes que no están presentes en todos los genomas,
#     basándose en un umbral definido.
#
#     Para cada bacteria:
#         - Calcula la frecuencia de presencia por familia.
#         - Separa familias core y no core.
#         - Guarda listas de familias no core.
#         - Filtra la matriz de pangenoma para conservar solo
#           las familias no core junto con las columnas base.
#
#
# Requiere:
#     - pandas
#     - os
##############################################################

## Familias  que no estan en el core 90%

In [1]:
import pandas as pd
import os

In [49]:
def analizar_core_familias(nombre_bacteria: str, porcentaje_umbral: float):
    """
    Analiza las familias de genes en un pangenoma y determina cuáles son core (comunes)
    y cuáles no, basándose en un umbral porcentual de presencia en genomas.

    Parámetros
    ----------
    nombre_bacteria : str
        Nombre de la bacteria (sin el sufijo "_merged").
    porcentaje_umbral : float
        Umbral mínimo de presencia (entre 0 y 1) requerido para considerar una familia como core.
        Por ejemplo, 0.95 significa "presente en al menos el 95% de los genomas".

    Estructura esperada
    -------------------
    - Entrada: 'merged/{nombre_bacteria}_merged.tsv'
    - Salidas:
        - 'frecuencias/frecuencia_familia_vs_genomas_{nombre_bacteria}.csv'
        - 'not_core/not_core_familias_{nombre_bacteria}.csv'
    """

    archivo = f"/files/atenea/CAMDA/2025/merged/{nombre_bacteria}_merged.tsv"
    if not os.path.exists(archivo):
        print(f" No se encontró el archivo: {archivo}")
        return

    matriz = pd.read_csv(archivo, sep="\t")

    # Seleccionar solo columnas de familias (desde la columna 8 en adelante)
    familias_df = matriz.iloc[:, 7:]

    # Número de genomas (columna 3, índice 2)
    total_genomas = matriz.iloc[:, 2].nunique()

    # Calcular frecuencia de cada familia (conteo de presencia > 0)
    frecuencia_por_familia = (familias_df > 0).sum(axis=0)

    frecuencia_df = pd.DataFrame({
        "familia": familias_df.columns,
        "frecuencia_genomas": frecuencia_por_familia.values
    })

    os.makedirs("frecuencias", exist_ok=True)
    os.makedirs("not_core", exist_ok=True)

    frecuencia_df.to_csv(f"frecuencias/frecuencia_familia_vs_genomas_{nombre_bacteria}.csv", index=False)

    umbral = int(porcentaje_umbral * total_genomas)

    # Clasificar familias
    not_core_familias = frecuencia_df[
        (frecuencia_df["frecuencia_genomas"] < umbral) & (frecuencia_df["frecuencia_genomas"] != 0)
    ]
    core_familias = frecuencia_df[frecuencia_df["frecuencia_genomas"] >= umbral]
    familias_sobrantes = (frecuencia_df["frecuencia_genomas"] == 0).sum()

    # Guardar las familias not-core
    not_core_familias[["familia"]].to_csv(f"not_core/not_core_familias_{nombre_bacteria}.csv", index=False)

    # Imprimir resumen
    print(f" {nombre_bacteria} — Umbral: {int(porcentaje_umbral * 100)}%")
    print(f" Familias core: {len(core_familias)}")
    print(f" Familias not-core: {len(not_core_familias)}")
    print(f" Familias ausentes en todos los genomas: {familias_sobrantes}")


## Cortar pangenomas para conservar solo las familias que estan en el no core

In [41]:
def filtrar_not_core(nombre_bacteria: str, porcentaje_umbral: float = 0.90):
    """
    Filtra del pangenoma solo las familias not-core (presentes en algunos pero no en todos los genomas)
    y elimina aquellas ausentes en todos.

    Parámetros
    ----------
    nombre_bacteria : str
        Nombre de la bacteria (ej. "neisseria").

    porcentaje_umbral : float
        Umbral para definir qué familias se consideran "core" (por defecto 0.90 = 90%).

    Requiere
    --------
    - Archivo TSV en `merged/{nombre_bacteria}_merged.tsv`.
    - Crea carpeta `pangenomas_filtrados/` si no existe.
    """

    archivo_pangenoma = f"/files/atenea/CAMDA/2025/merged/{nombre_bacteria}_merged.tsv"
    
    df = pd.read_csv(archivo_pangenoma, sep='\t')

    familias_df = df.iloc[:, 7:]  # solo las columnas de familias
    total_genomas = df.iloc[:, 2].nunique()

    # Calcular frecuencia de presencia por familia
    frecuencia = (familias_df > 0).sum(axis=0)

    # Definir core y not-core (excluir también las de frecuencia 0)
    umbral = int(porcentaje_umbral * total_genomas)
    columnas_not_core = frecuencia[(frecuencia < umbral) & (frecuencia > 0)].index.tolist()

    columnas_base = list(df.columns[:7])
    columnas_filtradas = columnas_base + columnas_not_core
    df_filtrado = df[columnas_filtradas]

    os.makedirs("pangenomas_filtrados", exist_ok=True)
    salida = f"pangenomas_filtrados/filtrado_{nombre_bacteria}.csv"
    df_filtrado.to_csv(salida, index=False)

    print(f" {nombre_bacteria} filtrado:")
    print(f" Familias not-core conservadas: {len(columnas_not_core)}")
    print(f" Familias eliminadas (core o ausentes): {familias_df.shape[1] - len(columnas_not_core)}\n")


In [28]:
analizar_core_familias("campylobacter", 0.90)  # 90% de los genomas

 campylobacter — Umbral: 90%
 Familias core: 1439
 Familias not-core: 11995
 Familias ausentes en todos los genomas: 0


/tmp/ipykernel_2415661/615954747.py:30: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  matriz = pd.read_csv(archivo, sep="\t")


In [43]:
filtrar_not_core("campylobacter", 0.90)

/tmp/ipykernel_2415661/3087308374.py:22: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo_pangenoma, sep='\t')


 campylobacter filtrado:
 Familias not-core conservadas: 11995
 Familias eliminadas (core o ausentes): 1439



In [29]:
analizar_core_familias("acinetobacter", 0.90)  # 90% de los genomas

/tmp/ipykernel_2415661/615954747.py:30: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  matriz = pd.read_csv(archivo, sep="\t")


 acinetobacter — Umbral: 90%
 Familias core: 2918
 Familias not-core: 66861
 Familias ausentes en todos los genomas: 4


In [44]:
filtrar_not_core("acinetobacter", 0.90)

/tmp/ipykernel_2415661/3087308374.py:22: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo_pangenoma, sep='\t')


 acinetobacter filtrado:
 Familias not-core conservadas: 66861
 Familias eliminadas (core o ausentes): 2922



In [32]:
analizar_core_familias("neisseria", 0.90)  # 90% de los genomas

/tmp/ipykernel_2415661/615954747.py:30: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  matriz = pd.read_csv(archivo, sep="\t")


 neisseria — Umbral: 90%
 Familias core: 2042
 Familias not-core: 13604
 Familias ausentes en todos los genomas: 1


In [42]:
filtrar_not_core("neisseria", 0.90)

/tmp/ipykernel_2415661/3087308374.py:22: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo_pangenoma, sep='\t')


 neisseria filtrado:
 Familias not-core conservadas: 13604
 Familias eliminadas (core o ausentes): 2043



In [31]:
analizar_core_familias("escherichia", 0.90)  # 90% de los genomas

/tmp/ipykernel_2415661/615954747.py:30: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  matriz = pd.read_csv(archivo, sep="\t")


 escherichia — Umbral: 90%
 Familias core: 3414
 Familias not-core: 163692
 Familias ausentes en todos los genomas: 6635


In [45]:
filtrar_not_core("escherichia", 0.90)

/tmp/ipykernel_2415661/3087308374.py:22: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo_pangenoma, sep='\t')


 escherichia filtrado:
 Familias not-core conservadas: 163692
 Familias eliminadas (core o ausentes): 10049



In [33]:
analizar_core_familias("pseudomonas", 0.90)  # 90% de los genomas

/tmp/ipykernel_2415661/615954747.py:30: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  matriz = pd.read_csv(archivo, sep="\t")


 pseudomonas — Umbral: 90%
 Familias core: 4366
 Familias not-core: 189486
 Familias ausentes en todos los genomas: 75


In [46]:
filtrar_not_core("pseudomonas", 0.90)

/tmp/ipykernel_2415661/3087308374.py:22: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo_pangenoma, sep='\t')


 pseudomonas filtrado:
 Familias not-core conservadas: 189486
 Familias eliminadas (core o ausentes): 4441



In [34]:
analizar_core_familias("salmonella", 0.90)  # 90% de los genomas

/tmp/ipykernel_2415661/615954747.py:30: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  matriz = pd.read_csv(archivo, sep="\t")


 salmonella — Umbral: 90%
 Familias core: 3802
 Familias not-core: 46000
 Familias ausentes en todos los genomas: 0


In [47]:
filtrar_not_core("salmonella", 0.90)

/tmp/ipykernel_2415661/3087308374.py:22: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo_pangenoma, sep='\t')


 salmonella filtrado:
 Familias not-core conservadas: 46000
 Familias eliminadas (core o ausentes): 3802



In [35]:
analizar_core_familias("klebsiella", 0.90)  # 90% de los genomas

/tmp/ipykernel_2415661/615954747.py:30: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  matriz = pd.read_csv(archivo, sep="\t")


 klebsiella — Umbral: 90%
 Familias core: 4330
 Familias not-core: 117305
 Familias ausentes en todos los genomas: 3097


In [48]:
filtrar_not_core("klebsiella", 0.90)

/tmp/ipykernel_2415661/3087308374.py:22: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo_pangenoma, sep='\t')


 klebsiella filtrado:
 Familias not-core conservadas: 117305
 Familias eliminadas (core o ausentes): 7427

